In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, time

## 1. Introduction & Dataset Overview  

ACCRE staff have hypothesized that the job scheduler sometimes becomes unresponsive because it is processing large bursts of job completions. This especially affects automated job submitters, such as members of the Open Science Grid.

This project is meant to evaluate whether the data supports the hypothesis of bursts of job completions contributing to scheduler unresponsiveness.

This project makes use of three datasets:  
* fullsample.csv: Contains slurm job records. Job completions correspond to jobs in the "COMPLETED" state with exit code "0:0".  
* slurm_wrapper_ce5.log, slurm_wrapper_ce6.log: These log files contain every slurm command executed by the CE5 and CE6 servers (gateways to the Open Science Grid).  
Unresponsive periods are indicated by "sbatch" commands from user 9204 that have:  
    * return code = 1
    * execution time > 15 seconds

## 2. Data Exploration & Cleaning  

### Full sample data (Slurm job records)

In [2]:
jobs = pd.read_csv("../data/fullsample.csv")
jobs.head(5)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


The fullsample dataset contains job records, with one row per job. Each job gets a unique ID, contained in the JOBID column.

Some jobs can be submitted as arrays of similar jobs. These are listed with an underscore in the JOBID, where the number after the underscore indicates the tasknumber.

In [3]:
print(f'We have data for {jobs.shape[0]:,} total jobs (or job parts).')

We have data for 7,395,885 total jobs (or job parts).


Description of job columns:

In [4]:
jobs.columns

Index(['JOBID', 'STATE', 'BEGIN', 'END', 'REQMEM', 'USEDMEM', 'REQTIME',
       'USEDTIME', 'NODES', 'CPUS', 'PARTITION', 'EXITCODE'],
      dtype='object')

| Column | Description |
|---|---|
| **JOBID** | The identification number of the job or job step. Array jobs are in the form ArrayJobID_ArrayTaskID.  | 
| **STATE** | Job state or status (COMPLETED, CANCELLED, FAILED, TIMEOUT, PREEMPTED, etc.). | 
| **BEGIN** | Beginning time for the job (may be different from the submit time). |
| **END** | Ending time for the job.     |
| **REQMEM** | Requested memory in megabytes. May be per-core (Mc) or per-node (Mn).   |
| **USEDMEM** | Used memory in megabytes per-node.   |
| **REQTIME** | Requested time in d-hh:mm:ss or hh:mm:ss.   |
| **USEDTIME** | Used time in d-hh:mm:ss or hh:mm:ss.   |
| **NODES** | Number of servers used for this job.   |
| **CPUS** | Total number of CPU-cores allocated to the job.   |
| **PARTITION** | Identifies the partition on which the job ran.    |
| **EXITCODE** | The exit code returned by the job script or salloc, typically as set by the exit() function. Following the colon is the signal that caused the process to terminate if it was terminated by a signal.    |

In [5]:
(jobs['JOBID'].value_counts()>1).sum()

np.int64(0)

We see that each row contains a unique job (or job part).

In [6]:
jobs['NODES'].value_counts().reset_index().sort_values(by='NODES')

,NODES,count
0,1,7385963
1,2,3525
2,3,2716
4,4,860
6,5,217
...,...,...
64,138,1
69,150,1
68,180,1
47,200,2


Most jobs ran on 1, 2, 3, or 8 nodes. There are a few outlier jobs that ran on 200 and 500 nodes.

In [7]:
jobs['PARTITION'].value_counts()

PARTITION
production              7019578
nogpfs                   147229
pascal                   124453
sam                       64967
turing                    21424
maxwell                   11278
cgw-maizie                 4309
debug                      1616
cgw-platypus                379
cgw-dsi-gw                  228
cgw-capra1                  157
cgw-dougherty1              125
cgw-horus                    61
cgw-cqs1                     28
cgw-hanuman                  21
cgw-sideshowbob              14
cgw-vm-qa-flatearth1          9
cgw-tbi01                     8
cgw-rocksteady                1
Name: count, dtype: int64

Most jobs ran on the "production", "nogpfs", or "pascal" partitions. The "debug" and "sam" partitions are test jobs that are expected to be short, and the "maxwell", "pascal", and "turing" partitions are for GPU resources.

In [8]:
jobs.groupby(['STATE','EXITCODE'])['JOBID'].count()

STATE          EXITCODE
CANCELLED      0:0         5378
               0:11          35
               0:15        1886
               0:40           1
               0:6           18
                           ... 
FAILED         9:0            3
NODE_FAIL      1:0            2
OUT_OF_MEMORY  0:125       1739
PENDING        0:0          443
RUNNING        0:0          208
Name: JOBID, Length: 227, dtype: int64

Most jobs are in the 'COMPLETED' state, with an exit code of 0:0. The next most common states are 'CANCELLED' and 'FAILED'. Also note that there are 208 running jobs and 443 pending jobs.

We also see many states of the form 'CANCELLED by \<number\>', where '\<number\>' might refer to a user ID.

In [9]:
jobs[jobs['STATE']=='RUNNING'].head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
7343,31874232_1949,RUNNING,2021-08-31T23:55:05,Unknown,2048Mn,0,00:15:00,36-20:34:17,1,1,production,0:0
7650,31874232_2461,RUNNING,2021-08-31T23:58:07,Unknown,2048Mn,144.11M,00:15:00,36-20:31:15,1,1,production,0:0
7676,31874232_2487,RUNNING,2021-08-31T23:58:07,Unknown,2048Mn,0,00:15:00,36-20:31:15,1,1,production,0:0
8621,31874232_3432,RUNNING,2021-09-01T00:03:07,Unknown,2048Mn,0,00:15:00,36-20:26:15,1,1,production,0:0


In [10]:
jobs[jobs['STATE']=='RUNNING']['END'].nunique()

1

Note that all 'RUNNING' jobs have an 'END' of 'Unknown'.

In [11]:
jobs[jobs['STATE']=='PENDING'].head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
830993,31940830,PENDING,Unknown,Unknown,251904Mn,0,13-18:00:00,00:00:00,1,16,production,0:0
923853,32808176,PENDING,Unknown,Unknown,81920Mn,0,3-00:00:00,00:00:00,1,2,production,0:0
923922,32907807_430,PENDING,Unknown,Unknown,16384Mn,0,2-02:00:00,00:00:00,1,1,production,0:0
923923,32907807_311,PENDING,Unknown,Unknown,16384Mn,0,2-02:00:00,00:00:00,1,1,production,0:0
923924,32907807_325,PENDING,Unknown,Unknown,16384Mn,0,2-02:00:00,00:00:00,1,1,production,0:0


In [12]:
jobs[jobs['STATE']=='PENDING'][['BEGIN','END','USEDTIME']].nunique()

BEGIN       1
END         1
USEDTIME    1
dtype: int64

Note that all 'PENDING' jobs have both a 'BEGIN' and an 'END' of 'Unknown' and a 'USEDTIME' of 00:00:00.

In [13]:
jobs[(jobs['BEGIN'] == 'Unknown') | (jobs['END'] == 'Unknown')].value_counts('STATE')

STATE
PENDING    443
RUNNING    208
Name: count, dtype: int64

Furthermore, we see that any job with 'Unknown' as the 'BEGIN' or 'END' is in either a 'PENDING' or 'RUNNING' state.

In [14]:
# Check data integrity for the 'USEDTIME' column. It should equal 'END' minus 'BEGIN'

# Create a new dataframe with just the begin, end, and usedtime times for jobs that have a begin and end time
job_times = jobs[(jobs['BEGIN'] != 'Unknown') & (jobs['END'] != 'Unknown')][['JOBID','BEGIN','END','USEDTIME']]

# Convert 'END' and 'BEGIN' to datetime
job_times['BEGIN'] = pd.to_datetime(job_times['BEGIN'], format = "%Y-%m-%dT%H:%M:%S")
job_times['END'] = pd.to_datetime(job_times['END'], format = "%Y-%m-%dT%H:%M:%S")

# Manually calculate time difference between 'BEGIN' and 'END' for each row and assign to a new 'USEDTIME_MANUAL' column
# Clean up the output by replacing ' days ' with a dash, and remove leading '0-' if it is present
usedtime_manual = [(str(diff)).replace(' days ', '-') for diff in (job_times['END'] - job_times['BEGIN'])]
usedtime_manual_cleaned = [value.replace('0-','') if value[:2]=='0-' else value for value in usedtime_manual]
job_times['USEDTIME_MANUAL'] = usedtime_manual_cleaned

# Find instances where the manually calculated usedtime matches the value of 'USEDTIME
usedtime_equ = job_times['USEDTIME_MANUAL'] == job_times['USEDTIME']

# Find any rows with inconsistent 'USEDTIME' values and filter them out of the data 
inconsistent_usedtime = job_times[~usedtime_equ]
inconsistent_usedtime

,JOBID,BEGIN,END,USEDTIME,USEDTIME_MANUAL
4186164,27050750_43,2021-03-06 19:39:30,2021-03-16 20:39:41,10-00:00:11,10-01:00:11
4186165,27050750_44,2021-03-07 18:54:30,2021-03-17 19:55:04,10-00:00:34,10-01:00:34
4186166,27050750_45,2021-03-07 18:54:30,2021-03-17 19:55:04,10-00:00:34,10-01:00:34
4186170,27050750_49,2021-03-10 01:45:37,2021-03-20 02:46:01,10-00:00:24,10-01:00:24
4186171,27050750_50,2021-03-11 07:42:19,2021-03-20 10:04:20,9-01:22:01,9-02:22:01
...,...,...,...,...,...
6257214,25494314,2020-11-01 01:38:54,2020-11-01 01:17:42,00:38:48,-1-+23:38:48
6257215,25494322,2020-11-01 01:37:07,2020-11-01 01:02:01,00:24:54,-1-+23:24:54
6257230,25494499,2020-11-01 01:49:02,2020-11-01 01:35:37,00:46:35,-1-+23:46:35
6257231,25494500,2020-11-01 01:49:02,2020-11-03 00:13:30,1-23:24:28,1-22:24:28


We see that some jobs have small discrepancies in the USEDTIME when compared to the value of the end time minus the begin time. Example:

In [23]:
inconsistent_usedtime[inconsistent_usedtime['JOBID']=='27050750_43']

,JOBID,BEGIN,END,USEDTIME,USEDTIME_MANUAL
4186164,27050750_43,2021-03-06 19:39:30,2021-03-16 20:39:41,10-00:00:11,10-01:00:11


Other jobs even have an end time that is before the begin time. Example:

In [24]:
inconsistent_usedtime[inconsistent_usedtime['JOBID']=='25494314']

,JOBID,BEGIN,END,USEDTIME,USEDTIME_MANUAL
6257214,25494314,2020-11-01 01:38:54,2020-11-01 01:17:42,00:38:48,-1-+23:38:48


### OSG Job Logs from CE5

In [26]:
ce5 = pd.read_csv("../data/slurm_wrapper_ce5.log",
                  header=None,
                  delimiter=' - ',
                  engine='python')
ce5.head(5)

,0,1,2,3,4,5
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
2,2020-10-16 08:22:48.128689,user 0,retry 0,time 0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
3,2020-10-16 08:25:13.257408,user 0,retry 0,time 0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
4,2020-10-16 08:31:01.460723,user 0,retry 0,time 0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."


### OSG Job Logs from CE6

In [27]:
ce6 = pd.read_csv("../data/slurm_wrapper_ce5.log",
                  header=None,
                  delimiter=' - ',
                  engine='python')
ce6.head(5)

,0,1,2,3,4,5
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
2,2020-10-16 08:22:48.128689,user 0,retry 0,time 0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
3,2020-10-16 08:25:13.257408,user 0,retry 0,time 0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
4,2020-10-16 08:31:01.460723,user 0,retry 0,time 0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."


## 3. Feature Engineering

We'll convert the date and time columns to a datetime data type.

In [15]:
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'], format = "%Y-%m-%dT%H:%M:%S", errors='coerce')
jobs['END'] = pd.to_datetime(jobs['END'], format = "%Y-%m-%dT%H:%M:%S", errors='coerce')

Next, we'll filter out all jobs that have inconsistent 'USEDTIME' values (e.g. the begin time is after the end time, or the USEDTIME value does not equal the end time minus the begin time).

In [16]:
jobs = jobs[~jobs['JOBID'].isin(inconsistent_usedtime['JOBID'].to_list())]

## 4. Analysis & Visualizations  

## 5. Interpretation & Conclusion